# Forest learning model for of the network

Here we want to find out that "is there any rule to be admitted in a community in the netwok?"
If community nodes have same criteria in a certain network it can be seen by tree splitting branches.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [2]:
import Analysis_Tools_Homans
import os
import sys

Version definition and path finder to begin evaluation

In [3]:
N = 100
T = 5000
version = 'p2_p1_p0'

OSNAME = {'win32':'\\', 'linux':'/'}
if sys.platform.startswith('win32'):
    plat = 'win32'
elif sys.platform.startswith('linux'):
    plat = 'linux'
current_path = os.getcwd()
path = current_path +OSNAME[plat]+'runned_files'+OSNAME[plat]+'N%d_T%d'%(N,T)+OSNAME[plat]+version+OSNAME[plat]

memory_size = 10
saving_time = T
path += '0_%d'%(T)+OSNAME[plat]

Load the data

In [4]:
dataframe = pd.read_csv(path + 'all_agents_properties.csv')
dataframe

,money,asset,approval,worth_ratio,situation,others_feeling,degree,community,self_value,value_to_others
0,6.619,11.170720,7.730,1.698259,0.026326,-0.024221,14.0,0.0,8.223174,10.626809
1,5.988,8.375439,2.419,1.013220,0.523865,0.022859,11.0,2.0,7.376878,6.493091
2,4.533,5.214230,0.350,0.513777,0.293733,-0.058866,5.0,6.0,5.556742,5.595191
3,4.973,12.296482,9.208,1.257325,0.858474,0.146288,15.0,5.0,12.461505,13.431610
4,5.568,7.300281,2.639,1.523425,0.883677,0.251954,11.0,6.0,13.594208,13.449662
...,...,...,...,...,...,...,...,...,...,...
95,6.914,11.677405,4.056,0.851492,0.119950,0.210008,7.0,2.0,6.949176,6.611934
96,5.477,14.725474,5.924,0.640538,0.022169,0.229055,7.0,0.0,7.524056,6.515401
97,2.469,8.104647,9.967,1.768563,0.367428,0.004050,16.0,2.0,6.546851,6.602232
98,14.168,15.029061,1.316,1.528347,0.300481,-0.023751,6.0,4.0,6.206700,6.300039


In [5]:
dataframe['community'].unique()

array([ 0.,  2.,  6.,  5.,  3.,  4.,  1., -1.])